In [1]:
#adapted from @bact at https://colab.research.google.com/drive/1hdtmwTXHLrqNmDhDqHnTQGpDVy1aJc4t
import json
import pandas as pd
import numpy as np
import re
import pycrfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pythainlp.tokenize import word_tokenize
from pythainlp.tag import pos_tag
with open('data/talks.transcript.th-en.24-10-2019_11:12.json', 'r') as f:
    data = json.load(f)

In [2]:
talk_idx = []
idx = []
th_phrases = []
en_phrases = []

for i in range(len(data)):
    nb_phrases = min(len(data[i]['en']),len(data[i]['th']))
    for j in range(nb_phrases):
        talk_idx.append(i)
        th_phrases.append(data[i]['th'][j])
        en_phrases.append(data[i]['en'][j])
        if data[i]['en'][j][-1]=='.':
            idx.append(1)
        else:
            idx.append(0)

In [3]:
phrase_df = pd.DataFrame({'talk_idx':talk_idx,'en_phrase':en_phrases ,'th_phrase':th_phrases, 'idx':idx})
phrase_df.idx.sum()

136463

In [4]:
all_sentences = []
for i in range(phrase_df.talk_idx.max()):
    df = phrase_df[phrase_df.talk_idx==i]
    sentences = []
    for j,row in df.iterrows():
        sentences.append(row.th_phrase)
        if row.idx==1:
            sentences.append(' |')
    joined_sentences = ''.join(sentences)
    #remove parantheses like (audience claps)
    joined_sentences = re.sub(r'\([^)]*\)', '', joined_sentences)
    #skip if talk is nothing but parantheses
    if joined_sentences=='': continue
    #remove | at the last sentence if present
    if joined_sentences[-1]=='|': joined_sentences = joined_sentences[:-1]
    all_sentences.append(joined_sentences)

In [5]:
len(all_sentences)

1544

In [6]:
all_sentences[0]

'บนหลังอาชาแห้งกร่องดอนกิโฆเต้ พระเอกของเราบุกตะลุยสู้กับกองทัพยักษ์ |ในสายตาของเขา มันเป็นหน้าที่ของเขาที่จะปราบอสูรร้ายเหล่านี้ในนามแห่งหญิงอันเป็นที่รักของเขา ดุลสิเนอา |ทว่า การกระทำอันหาญกล้านี้ก็สูญเปล่า |เมื่อซานโซ่ ปันซ่า ผู้รับใช้ของเขาอธิบายครั้งแล้วครั้งเล่า ว่าสิ่งเหล่านี้จะเป็นยักษ์ก็หาไม่พวกมันเป็นเพียงกังหันลมเท่านั้น |ดอนกิโฆเต้ หาได้เสียความแน่วแน่แทงทวนของเขาเข้าไปยังใบพัดอย่างจัง |ด้วยพลังใจที่ไม่เคยถดถอยอัศวินผู้นั้นยืนขึ้นอย่างภาคภูมิและยิ่งเชื่อมั่นในปฏิบัติการของเขามากขึ้น |ลำดับเหตุการณ์นี้ครอบคลุมเรื่องราวส่วนใหญ่ของดอนกิโฆเต้ ที่เป็นที่รักมหากาพย์ ไร้ตรรกะ และมีชีวิตชีวาของ อลองโซ กีฆานาผู้กลายเป็น ดอนกิโฆเต้ แห่งลามันช่าผู้ซุ่มซ่ามแต่กล้าหาญหรือที่รู้จักกันในนาม ขุนนางต่ำศักดิ์นักฝัน |แต่เดิมวรรณกรรมนี้มีสองเล่มบรรยายเรื่องราวของดอนกิโฆเต้ในขณะที่เขาเดินทางผ่านตอนกลาง และตอนเหนือของสเปนเพื่อต่อสู้กับบรรดาปิศาจร้าย |แม้จินตนาการในเรื่อง ดอนกิโฆเต้ จะสูงล้ำเหนือเมฆผู้ประพันธ์ มิเกล เด เซร์บันเตสก็ไม่เคยนึกฝันว่าหนังสือของเขาจะกลายเป็นนิยายที่ขายดีที่สุดตลอดกาล 

In [7]:
all_tuples = []
for i in range(len(all_sentences)):
    tuples = []
    for s in all_sentences[i].split('|'):
        s_lst = word_tokenize(s)
        for j in range(len(s_lst)):
            lab = 'B' if j==0 else 'I'
            tuples.append((s_lst[j],lab))
    all_tuples.append(tuples)

In [8]:
len(all_tuples)

1544

In [11]:
def extract_features(doc, window=2, max_n_gram = 3):
    doc_features = []
    doc_pos = ['xxpad' for i in range(window)] + [p for (w,p) in pos_tag(doc)] + ['xxpad' for i in range(window)]
    doc = ['xxpad' for i in range(window)] + doc + ['xxpad' for i in range(window)]
    for i in range(window, len(doc)-window):
        word_features = ['bias']
        for n_gram in range(1, min(max_n_gram+1,2+window*2)):
            for j in range(i-window,i+window+2-n_gram):
                feature_position = f'{n_gram}_{j-i}_{j-i+n_gram}'
                word_ = f'{"|".join(doc[j:(j+n_gram)])}'
                pos_ =f'{"|".join(doc_pos[j:(j+n_gram)])}'
                word_features += [f'word_{feature_position}={word_}']
                word_features += [f'pos_{feature_position}={pos_}']
        doc_features.append(word_features)
    return doc_features

extract_features(word_tokenize('ฉันชอบกินมะนาว'), window=2, max_n_gram = 3)

In [13]:
#target
y = [[l for (w,l) in t] for t in all_tuples]
#features
x_pre = [[w for (w,l) in t] for t in all_tuples]
x = [extract_features(x_, window=2, max_n_gram = 3) for x_ in x_pre]

In [14]:
len(x),len(y)

(1544, 1544)

In [15]:
# Split train and test set at 80/20 proportion
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1412)

In [16]:
x_train[0][:5]

[['bias',
  'word_1_-2_-1=xxpad',
  'pos_1_-2_-1=xxpad',
  'word_1_-1_0=xxpad',
  'pos_1_-1_0=xxpad',
  'word_1_0_1=วงดนตรี',
  'pos_1_0_1=NCMN',
  'word_1_1_2=ที่',
  'pos_1_1_2=PREL',
  'word_1_2_3=คุณ',
  'pos_1_2_3=VACT',
  'word_2_-2_0=xxpad|xxpad',
  'pos_2_-2_0=xxpad|xxpad',
  'word_2_-1_1=xxpad|วงดนตรี',
  'pos_2_-1_1=xxpad|NCMN',
  'word_2_0_2=วงดนตรี|ที่',
  'pos_2_0_2=NCMN|PREL',
  'word_2_1_3=ที่|คุณ',
  'pos_2_1_3=PREL|VACT',
  'word_3_-2_1=xxpad|xxpad|วงดนตรี',
  'pos_3_-2_1=xxpad|xxpad|NCMN',
  'word_3_-1_2=xxpad|วงดนตรี|ที่',
  'pos_3_-1_2=xxpad|NCMN|PREL',
  'word_3_0_3=วงดนตรี|ที่|คุณ',
  'pos_3_0_3=NCMN|PREL|VACT'],
 ['bias',
  'word_1_-2_-1=xxpad',
  'pos_1_-2_-1=xxpad',
  'word_1_-1_0=วงดนตรี',
  'pos_1_-1_0=NCMN',
  'word_1_0_1=ที่',
  'pos_1_0_1=PREL',
  'word_1_1_2=คุณ',
  'pos_1_1_2=VACT',
  'word_1_2_3=ชื่นชอบ',
  'pos_1_2_3=VACT',
  'word_2_-2_0=xxpad|วงดนตรี',
  'pos_2_-2_0=xxpad|NCMN',
  'word_2_-1_1=วงดนตรี|ที่',
  'pos_2_-1_1=NCMN|PREL',
  'word_2_0_2=ที่

In [17]:
# Train model
trainer = pycrfsuite.Trainer(verbose=True)

for xseq, yseq in zip(x_train, y_train):
  trainer.append(xseq, yseq)

trainer.set_params({
    'c1': 1,
    'c2': 1e-1,
    'max_iterations': 200,
    'feature.possible_transitions': True,
})

trainer.train('sentenceseg-crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 7172373
Seconds required: 98.255

L-BFGS optimization
c1: 1.000000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 456579.138411
Feature norm: 1.000000
Error norm: 243524.282175
Active features: 991989
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 22.741

***** Iteration #2 *****
Loss: 439842.568227
Feature norm: 1.052375
Error norm: 219713.654795
Active features: 735487
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 9.687

***** Iteration #3 *****
Loss: 338252.549844
Feature norm: 1.476930
Error norm: 207708.328002
Active features: 300388
Line search trials: 1
Line search step: 1.000000
Seconds

***** Iteration #40 *****
Loss: 121965.195730
Feature norm: 61.719729
Error norm: 5396.014034
Active features: 99717
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.073

***** Iteration #41 *****
Loss: 121644.489844
Feature norm: 64.110463
Error norm: 2308.764474
Active features: 98521
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.977

***** Iteration #42 *****
Loss: 121342.487759
Feature norm: 66.637052
Error norm: 2869.542036
Active features: 96931
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.083

***** Iteration #43 *****
Loss: 121073.563100
Feature norm: 68.861126
Error norm: 2410.063096
Active features: 91302
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 6.934

***** Iteration #44 *****
Loss: 120865.959467
Feature norm: 70.702010
Error norm: 2210.147840
Active features: 90695
Line search trials: 1
Line search step: 1.0

***** Iteration #80 *****
Loss: 119160.941139
Feature norm: 85.014125
Error norm: 1392.391433
Active features: 72349
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.153

***** Iteration #81 *****
Loss: 119155.887454
Feature norm: 85.052260
Error norm: 1235.806081
Active features: 72233
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.360

***** Iteration #82 *****
Loss: 119150.634987
Feature norm: 85.095857
Error norm: 979.812910
Active features: 72168
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.183

***** Iteration #83 *****
Loss: 119146.182571
Feature norm: 85.119142
Error norm: 1018.481464
Active features: 72013
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.007

***** Iteration #84 *****
Loss: 119142.198709
Feature norm: 85.155741
Error norm: 1322.202921
Active features: 71918
Line search trials: 1
Line search step: 1.00

***** Iteration #124 *****
Loss: 119055.418400
Feature norm: 85.902600
Error norm: 941.289299
Active features: 70557
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.035

***** Iteration #125 *****
Loss: 119053.697400
Feature norm: 85.914482
Error norm: 741.670836
Active features: 70540
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.622

***** Iteration #126 *****
Loss: 119052.669407
Feature norm: 85.930355
Error norm: 962.922947
Active features: 70531
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.939

***** Iteration #127 *****
Loss: 119050.942044
Feature norm: 85.938921
Error norm: 776.801639
Active features: 70500
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 7.592

***** Iteration #128 *****
Loss: 119049.947306
Feature norm: 85.956215
Error norm: 981.417900
Active features: 70478
Line search trials: 1
Line search step: 1.0

***** Iteration #165 *****
Loss: 119011.262350
Feature norm: 86.307591
Error norm: 591.390292
Active features: 69784
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.822

***** Iteration #166 *****
Loss: 119010.659528
Feature norm: 86.316803
Error norm: 725.765170
Active features: 69782
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 9.450

***** Iteration #167 *****
Loss: 119009.678988
Feature norm: 86.321717
Error norm: 509.518354
Active features: 69775
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.358

***** Iteration #168 *****
Loss: 119009.165291
Feature norm: 86.328019
Error norm: 641.995562
Active features: 69784
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 8.151

***** Iteration #169 *****
Loss: 119008.261504
Feature norm: 86.334031
Error norm: 462.910771
Active features: 69743
Line search trials: 1
Line search step: 1.0

In [18]:
# Predict (using test set)
tagger = pycrfsuite.Tagger()
tagger.open('sentenceseg-crf.model')
y_pred = [tagger.tag(xseq) for xseq in x_test]

In [19]:
# Evaluate
labels = {'B': 0, "I": 1} # classification_report() needs values in 0s and 1s
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

print(classification_report(
    truths, predictions,
    target_names=["B", "I"]))

              precision    recall  f1-score   support

           B       0.73      0.70      0.71     27763
           I       0.99      0.99      0.99    649564

    accuracy                           0.98    677327
   macro avg       0.86      0.84      0.85    677327
weighted avg       0.98      0.98      0.98    677327



In [23]:
def sentence_tokenize(s):
    toks = word_tokenize(s)
    feat = extract_features(toks)
    labs = tagger.tag(feat)
    sentences = []
    sentence = ''
    for i, w in enumerate(toks):
        if labs[i] == 'B':
            if sentence:
                sentences.append(sentence)
            sentence = ''
        sentence = sentence + w
    if sentence:
        sentences.append(sentence)
    return sentences

In [24]:
s = 'เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์ เธอสามารถตัดสินใจด้วยตัวเอง แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์ ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก 30 ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์ แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ จะพร้อมเสียก่อน'

In [25]:
sentence_tokenize(s)

['เธอคือหุ่นยนต์รูปแบบใหม่ที่ฉันคนนี้สร้างขึ้นมาเธอมีความสามารถในการคิดและรู้สึกเหมือนมนุษย์<space>',
 'เธอสามารถตัดสินใจด้วยตัวเอง<space>',
 'แต่ถึงอย่างนั้นมันก็เป็นดาบสองคม<space>ที่อาจเป็นอันตรายถ้าเธอคิดว่ามนุษย์เป็นภัย<space>จนแหกกฎข้อแรกที่หุ่นยนต์ห้ามทำร้ายมนุษย์<space>',
 'ฉันจำเป็นต้องใช้เวลาทดสอบระบบอีก<space>30<space>ปีเพื่อความมั่นใจว่าเธอจะไม่เป็นอันตรายกับมนุษย์<space>',
 'แต่ฉันคงจะมีชีวิตอยู่ไม่ถึงวันนั้น<space>',
 'ดังนั้นฉันจึงปิดผนึกเธอเอาไว้ในแคปซูลจนกว่าระบบต่าง ๆ<space>จะพร้อมเสียก่อน']